In [1]:
# imports
import pandas as pd
import numpy as np
import joblib

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix

# Import SMOTE
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline

print("Libraries imported successfully.")

CLEANED_DATA_PATH = 'cleaned_diabetic_data.csv'

try:
    df = pd.read_csv(CLEANED_DATA_PATH)
    print(f"Loaded cleaned data. Shape: {df.shape}")
except FileNotFoundError:
    print(f"Error: '{CLEANED_DATA_PATH}' not found. Please check the file path.")
    # Stop execution if file isn't found
    raise

TARGET_COLUMN = 'target'

COLS_TO_DROP_FOR_MODELING = ['readmitted', 'diag_1', 'diag_2', 'diag_3']

try:
    y = df[TARGET_COLUMN]
    X = df.drop([TARGET_COLUMN] + COLS_TO_DROP_FOR_MODELING, axis=1)
    print("Features (X) and Target (y) are defined.")
    print(f"X shape: {X.shape}, y shape: {y.shape}")

except KeyError as e:
    print(f"Error: A required column is missing. {e}")
    print("Please ensure your cleaned data contains 'target' and the original 'diag' columns.")
    raise


numerical_cols = X.select_dtypes(include=np.number).columns
categorical_cols = X.select_dtypes(include=['object', 'category']).columns

print(f"\nIdentified {len(numerical_cols)} numerical features.")
print(f"Identified {len(categorical_cols)} categorical features.")

numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

#preprocessor pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ],
    remainder='passthrough'
)

# split before oversampling to preserve test set
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print(f"\nData split into training and testing sets:")
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")
print(f"Class 1 (Readmitted) in y_train: {y_train.sum()} ({y_train.mean()*100:.2f}%)")
print(f"Class 1 (Readmitted) in y_test:  {y_test.sum()} ({y_test.mean()*100:.2f}%)")


# initialize random forest classifier
rf_model = RandomForestClassifier(
    n_estimators=100,
    random_state=42,
    n_jobs=-1
)

# imblearn pipeline with SMOTE
pipeline = ImbPipeline(steps=[
    ('preprocessor', preprocessor),
    ('smote', SMOTE(random_state=42)),
    ('model', rf_model)
])

print("\nStarting Model Training (with SMOTE)")
# fit the entire pipeline on the training data
pipeline.fit(X_train, y_train)
print("Model Training Complete")

print("\nModel Evaluation (on Unseen Test Set)")

# make predictions on the original, non-resampled test set
y_pred = pipeline.predict(X_test)
y_pred_proba = pipeline.predict_proba(X_test)[:, 1] # Probabilities for class 1

print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=['Not Readmitted (0)', 'Readmitted (1)']))

# print the AUC-ROC score
auc_score = roc_auc_score(y_test, y_pred_proba)
print(f"AUC-ROC Score: {auc_score:.4f}")

# confusion matrix
print("\nConfusion Matrix")
cm = confusion_matrix(y_test, y_pred)
print("                 Predicted 0   |   Predicted 1")
print(f"Actual 0:    {cm[0,0]:>10}    |    {cm[0,1]:>10}  (False Positives)")
print(f"Actual 1:    {cm[1,0]:>10}    |    {cm[1,1]:>10}  (True Positives)")
print(f"\nKey Metric (Recall): The model correctly identified {cm[1,1]} out of {cm[1,0] + cm[1,1]} actual readmissions.")

# save trained pipeline
pipeline_filename = 'rf_smote_pipeline.joblib'
joblib.dump(pipeline, pipeline_filename)
print(f"\nSuccessfully saved the full pipeline to '{pipeline_filename}'")

X_test.to_csv('X_test.csv', index=False)
y_test.to_csv('y_test.csv', index=False)
print("Successfully saved 'X_test.csv' and 'y_test.csv' for the visualization notebook.")

Libraries imported successfully.
Loaded cleaned data. Shape: (67113, 46)
Features (X) and Target (y) are defined.
X shape: (67113, 41), y shape: (67113,)

Identified 11 numerical features.
Identified 30 categorical features.

Data split into training and testing sets:
X_train shape: (53690, 41), y_train shape: (53690,)
X_test shape: (13423, 41), y_test shape: (13423,)
Class 1 (Readmitted) in y_train: 4888 (9.10%)
Class 1 (Readmitted) in y_test:  1222 (9.10%)

Starting Model Training (with SMOTE)
Model Training Complete

Model Evaluation (on Unseen Test Set)
Classification Report:
                    precision    recall  f1-score   support

Not Readmitted (0)       0.91      1.00      0.95     12201
    Readmitted (1)       0.10      0.00      0.00      1222

          accuracy                           0.91     13423
         macro avg       0.51      0.50      0.48     13423
      weighted avg       0.84      0.91      0.87     13423

AUC-ROC Score: 0.6068

Confusion Matrix
          

In [3]:
# imports
import pandas as pd
import numpy as np
import joblib

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix

print("Libraries imported successfully.")

CLEANED_DATA_PATH = 'cleaned_diabetic_data.csv'
try:
    df = pd.read_csv(CLEANED_DATA_PATH)
    print(f"Loaded cleaned data. Shape: {df.shape}")
except FileNotFoundError:
    print(f"Error: '{CLEANED_DATA_PATH}' not found.")
    raise

TARGET_COLUMN = 'target'
COLS_TO_DROP_FOR_MODELING = ['readmitted', 'diag_1', 'diag_2', 'diag_3']

try:
    y = df[TARGET_COLUMN]
    X = df.drop([TARGET_COLUMN] + COLS_TO_DROP_FOR_MODELING, axis=1)
    print(f"Features (X) and Target (y) are defined. X shape: {X.shape}")
except KeyError as e:
    print(f"Error: A required column is missing. {e}")
    raise

numerical_cols = X.select_dtypes(include=np.number).columns
categorical_cols = X.select_dtypes(include=['object', 'category']).columns

print(f"Identified {len(numerical_cols)} numerical features.")
print(f"Identified {len(categorical_cols)} categorical features.")

numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])
categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ],
    remainder='passthrough'
)

# split
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)
print(f"Data split into training and testing sets.")

# standard sklearn pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', RandomForestClassifier(random_state=42, n_jobs=-1))
])

print("\nModel Pipeline Created")


# define the grid of parameters to search
# testing a few key parameters
param_grid = {
    'model__n_estimators': [100, 150],
    'model__max_depth': [10, 20],
    'model__min_samples_leaf': [5, 10],
    'model__class_weight': ['balanced', 'balanced_subsample']
}

# initialize gridsearch
grid_search = GridSearchCV(
    pipeline,
    param_grid,
    cv=3,
    scoring='recall',
    n_jobs=-1,
    verbose=2
)

print("\nStarting Hyperparameter Tuning (GridSearchCV)")
print("This will take several minutes")

# fit the grid search on the training data
grid_search.fit(X_train, y_train)

print("Tuning Complete")
print(f"Best parameters found: {grid_search.best_params_}")
print(f"Best recall score (on validation data): {grid_search.best_score_:.4f}")

best_model = grid_search.best_estimator_

print("\nEvaluation of best Model (on Unseen Test Set)")

y_pred = best_model.predict(X_test)
y_pred_proba = best_model.predict_proba(X_test)[:, 1] # Probabilities for class 1

# print the classification report
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=['Not Readmitted (0)', 'Readmitted (1)']))

# print the AUC-ROC score
auc_score = roc_auc_score(y_test, y_pred_proba)
print(f"AUC-ROC Score: {auc_score:.4f}")

print("\nConfusion Matrix")
cm = confusion_matrix(y_test, y_pred)
print("                 Predicted 0   |   Predicted 1")
print(f"Actual 0:    {cm[0,0]:>10}    |    {cm[0,1]:>10}  (False Positives)")
print(f"Actual 1:    {cm[1,0]:>10}    |    {cm[1,1]:>10}  (True Positives)")

recall = cm[1,1] / (cm[1,0] + cm[1,1])
print(f"\nKey Metric (Recall): The model correctly identified {cm[1,1]} out of {cm[1,0] + cm[1,1]}"
      f" actual readmissions. (Recall = {recall:.2%})")

# save the entire pipeline
pipeline_filename = 'rf_tuned_pipeline.joblib'
joblib.dump(best_model, pipeline_filename)
print(f"\nSuccessfully saved the tuned pipeline to '{pipeline_filename}'")

# save the test seta
X_test.to_csv('X_test.csv', index=False)
y_test.to_csv('y_test.csv', index=False)
#print("Successfully saved 'X_test.csv' and 'y_test.csv' for the visualization notebook.")

Libraries imported successfully.
Loaded cleaned data. Shape: (67113, 46)
Features (X) and Target (y) are defined. X shape: (67113, 41)
Identified 11 numerical features.
Identified 30 categorical features.
Data split into training and testing sets.

Model Pipeline Created

Starting Hyperparameter Tuning (GridSearchCV)
This will take several minutes
Fitting 3 folds for each of 16 candidates, totalling 48 fits
Tuning Complete
Best parameters found: {'model__class_weight': 'balanced_subsample', 'model__max_depth': 10, 'model__min_samples_leaf': 10, 'model__n_estimators': 150}
Best recall score (on validation data): 0.4673

Evaluation of best Model (on Unseen Test Set)
Classification Report:
                    precision    recall  f1-score   support

Not Readmitted (0)       0.93      0.69      0.79     12201
    Readmitted (1)       0.14      0.49      0.21      1222

          accuracy                           0.67     13423
         macro avg       0.53      0.59      0.50     13423
  

In [4]:
# imports
import pandas as pd
import numpy as np
import joblib
import xgboost as xgb

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix

# print("Libraries imported successfully.")

# loading data
df = pd.read_csv('cleaned_diabetic_data.csv')
y = df['target']
X = df.drop(['target', 'readmitted', 'diag_1', 'diag_2', 'diag_3'], axis=1)

# preprocessing
numerical_cols = X.select_dtypes(include=np.number).columns
categorical_cols = X.select_dtypes(include=['object', 'category']).columns

numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])
categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ],
    remainder='passthrough'
)

# calculate weights for imbalance
# formula: sum(negative instances) / sum(positive instances)
# this tells XGBoost to treat the minority class with x times more weight
scale_weight = (y == 0).sum() / (y == 1).sum()
print(f"Calculated scale_pos_weight: {scale_weight:.2f}")

# split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# build XGBoost pipeline
xgb_model = xgb.XGBClassifier(
    objective='binary:logistic',
    scale_pos_weight=scale_weight, # handles the imbalance
    random_state=42,
    n_jobs=-1,
    eval_metric='logloss'
)

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', xgb_model)
])

# hyperparameter tuning
# key hyperparameter selection
param_grid = {
    'model__n_estimators': [100, 200],
    'model__max_depth': [3, 5, 7],
    'model__learning_rate': [0.01, 0.1],
    'model__subsample': [0.8, 1.0]
}

print("\nStarting XGBoost Grid Search (F1 Score)")
grid_search = GridSearchCV(
    pipeline,
    param_grid,
    cv=3,
    scoring='f1',
    n_jobs=-1,
    verbose=2
)
grid_search.fit(X_train, y_train)

print(f"Best params: {grid_search.best_params_}")
best_xgb = grid_search.best_estimator_

# model evaluation
print("\nXGBoost Evaluation")
y_pred = best_xgb.predict(X_test)
y_pred_proba = best_xgb.predict_proba(X_test)[:, 1]

print(classification_report(y_test, y_pred))
print(f"AUC-ROC Score: {roc_auc_score(y_test, y_pred_proba):.4f}")

#confusion matrix
print("\nConfusion Matrix")
cm = confusion_matrix(y_test, y_pred)
print("                 Predicted 0   |   Predicted 1")
print(f"Actual 0:    {cm[0,0]:>10}    |    {cm[0,1]:>10}  (False Positives)")
print(f"Actual 1:    {cm[1,0]:>10}    |    {cm[1,1]:>10}  (True Positives)")

recall = cm[1,1] / (cm[1,0] + cm[1,1])
print(f"\nKey Metric (Recall): The model correctly identified {cm[1,1]} out of {cm[1,0] + cm[1,1]}"
      f" actual readmissions. (Recall = {recall:.2%})")

# Save the model
joblib.dump(best_xgb, 'xgb_tuned_pipeline.joblib')
print("XGBoost model saved.")
y_pred_proba = best_xgb.predict_proba(X_test)[:, 1]

# find the exact threshold for 70% recall
from sklearn.metrics import precision_recall_curve
import numpy as np

# calculate precision and recall for all possible thresholds
precisions, recalls, thresholds = precision_recall_curve(y_test, y_pred_proba)

# we want the threshold where recall is closest to 0.70
target_recall = 0.70

# find the index of the recall value closest to 0.70
optimal_idx = np.argmin(np.abs(recalls - target_recall))
optimal_threshold = thresholds[optimal_idx]

print(f"Threshold Tuning Results")
print(f"To achieve ~{target_recall*100}% Recall, the threshold moves from 0.50 to: {optimal_threshold:.4f}")

# applying new threshold
y_pred_optimized = (y_pred_proba >= optimal_threshold).astype(int)

# evaluation
from sklearn.metrics import classification_report, confusion_matrix

print("\n--- Optimized Classification Report (Threshold: {:.4f}) ---".format(optimal_threshold))
print(classification_report(y_test, y_pred_optimized, target_names=['Not Readmitted (0)', 'Readmitted (1)']))

#confusion matrix
print("\nConfusion Matrix")
cm = confusion_matrix(y_test, y_pred_optimized)
print("                 Predicted 0   |   Predicted 1")
print(f"Actual 0:    {cm[0,0]:>10}    |    {cm[0,1]:>10}  (False Positives)")
print(f"Actual 1:    {cm[1,0]:>10}    |    {cm[1,1]:>10}  (True Positives)")

recall = cm[1,1] / (cm[1,0] + cm[1,1])
print(f"\nKey Metric (Recall): The model correctly identified {cm[1,1]} out of {cm[1,0] + cm[1,1]}"
      f" actual readmissions. (Recall = {recall:.2%})")

Calculated scale_pos_weight: 9.98

Starting XGBoost Grid Search (F1 Score)
Fitting 3 folds for each of 24 candidates, totalling 72 fits
Best params: {'model__learning_rate': 0.1, 'model__max_depth': 3, 'model__n_estimators': 200, 'model__subsample': 0.8}

XGBoost Evaluation
              precision    recall  f1-score   support

           0       0.94      0.67      0.78     12201
           1       0.14      0.54      0.22      1222

    accuracy                           0.65     13423
   macro avg       0.54      0.60      0.50     13423
weighted avg       0.86      0.65      0.73     13423

AUC-ROC Score: 0.6385

Confusion Matrix
                 Predicted 0   |   Predicted 1
Actual 0:          8117    |          4084  (False Positives)
Actual 1:           563    |           659  (True Positives)

Key Metric (Recall): The model correctly identified 659 out of 1222 actual readmissions. (Recall = 53.93%)
XGBoost model saved.
Threshold Tuning Results
To achieve ~70.0% Recall, the thre

In [6]:
# optimal threshold search
from sklearn.metrics import precision_recall_curve

# using old probabilities
y_pred_proba = best_xgb.predict_proba(X_test)[:, 1]

# calculate precision-recall curve
precisions, recalls, thresholds = precision_recall_curve(y_test, y_pred_proba)

# find the threshold that gives us at least 70% recall
target_recall = 0.70
idx = np.argmax(recalls <= target_recall)
optimal_threshold = thresholds[idx]

print(f"To achieve {target_recall*100}% Recall, set threshold to: {optimal_threshold:.4f}")

# apply this new threshold
y_pred_optimized = (y_pred_proba >= optimal_threshold).astype(int)

# see the new result
print("\n--- Optimized Threshold Results ---")
print(classification_report(y_test, y_pred_optimized))

# confusion matrix
print("\nConfusion Matrix")
cm = confusion_matrix(y_test, y_pred)
print("                 Predicted 0   |   Predicted 1")
print(f"Actual 0:    {cm[0,0]:>10}    |    {cm[0,1]:>10}  (False Positives)")
print(f"Actual 1:    {cm[1,0]:>10}    |    {cm[1,1]:>10}  (True Positives)")

recall = cm[1,1] / (cm[1,0] + cm[1,1])
print(f"\nKey Metric (Recall): The model correctly identified {cm[1,1]} out of {cm[1,0] + cm[1,1]}"
      f" actual readmissions. (Recall = {recall:.2%})")

To achieve 70.0% Recall, set threshold to: 0.4396

--- Optimized Threshold Results ---
              precision    recall  f1-score   support

           0       0.94      0.48      0.64     12201
           1       0.12      0.70      0.20      1222

    accuracy                           0.50     13423
   macro avg       0.53      0.59      0.42     13423
weighted avg       0.87      0.50      0.60     13423


Confusion Matrix
                 Predicted 0   |   Predicted 1
Actual 0:          8117    |          4084  (False Positives)
Actual 1:           563    |           659  (True Positives)

Key Metric (Recall): The model correctly identified 659 out of 1222 actual readmissions. (Recall = 53.93%)
